# Libraries

In [1]:
import EDF_wrapper
import annotations_validation as av
import annotations_extraction as ae
import filters
import pyedflib
from pathlib import Path
import pandas as pd

In [ ]:
directory = Path("data/annotated/")
files = EDF_wrapper.read_files_from_dir(directory, load_files=True)
len(files)

ae.add_swallow_annotations_to_files(files)

In [3]:
ae.get_directory_features_excel_files(files)

The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 6/6 [00:17<00:00,  3.00s/it]


Wrong Probe Annotation Format s_klein_oblong_startt_<label>_start/stop required
Wrong Probe Annotation Format s_klein_oblong_stopt_<label>_start/stop required
The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 6/6 [00:24<00:00,  4.14s/it]


There are 1 annotations out of the pattern.
The annotations dataframe is empty.


...

Feature Extraction: 100%|██████████| 6/6 [00:22<00:00,  3.72s/it]

The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 3/3 [00:09<00:00,  3.03s/it]

The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 6/6 [00:12<00:00,  2.03s/it]


---

### Unit tests

In [ ]:
ann_file = files[1]  
ann_file["header"]["annotations"] = list(map(lambda x: [x[0], x[1], x[2].lower()], ann_file["header"]["annotations"]))
ann_file["header"]["annotations"]

In [ ]:
general_df = ae.create_annotations_df(ann_file, 'general')
swallows_df = ae.create_annotations_df(ann_file, 'swallows')

In [ ]:
df = ae.extract_features_from_annotations(swallows_df)
df

In [ ]:
df.shape

In [ ]:
df.to_excel(f"data/xlsx/{Path(ann_file['filepath']).stem}.xlsx", index=False)

### To-do:
- EMG_start = BI_start - 0,366 s